In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
# importamos los modelos que necesitemos
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from math import sqrt


from bootcampviztools import pinta_distribucion_categoricas, plot_categorical_numerical_relationship, plot_grouped_histograms


# 1. Procesado tratamiento datos

En nuestra checklist:

* Objetivo de negocio:
* Objetivo tecnico:
* Tipo de modelado (supervisado, no supervisado): 
* Features:
* Target, si hay (variable a predecir): 
* Tipo de problema (clasificación, regresión, etc): 
* Métrica de Evaluacion:
* Separación Train-Test:

In [ ]:
# separación train-test
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

# 2. Proceso EDA

In [ ]:
# convertir variable numérica a categórica
train_set["nombre_nueva_columna_categorica"] = pd.cut(train_set["columna_numerica"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

# análisis univariante numéricas (histogramas solo de numéricas)
train_set.hist(bins=50, figsize=(12, 8))
plt.show()

# análisis univariante categóricas
pinta_distribucion_categoricas(train_set, ["columas categoricas"], relativa= True, mostrar_valores= True)

# análisis multivariante categóricas vs target numerica
plot_categorical_numerical_relationship(train_set, categorical_col= "ocean_proximity", numerical_col="target")
plot_categorical_numerical_relationship(train_set, categorical_col= "income_cat", numerical_col="target")

train_set.plot(kind="scatter", x="longitude", y="latitude", grid=True,
             s=train_set["population"] / 100, label="population",
             c="target", cmap="jet", colorbar=True,
             legend=True, sharex=False, figsize=(10, 7))
plt.show()

# análisis multivariante numéricas vs target numerica
corr_matrix = train_set.corr(numeric_only= True)
corr_matrix["target"].sort_values(ascending = False)
columnas = corr_matrix["target"][corr_matrix["target"] > 0.07].index.to_list() # nos quedamos con las que tienen correlación alta
sns.pairplot(train_set[columnas]);

# analisis multivariante numericas vs target categorica
sns.pairplot(
    train_set[columnas_numericas],
    hue="Clicked on Ad", # variable target va a distinguir por colores
    diag_kind="kde",
    plot_kws={"alpha": 0.6}
)

# creamos otras variables y volvemos a ver correlación

# por último, hacemos nuestra lista de features con todo lo concluido.



# 3. Tratamiento categóricas

In [ ]:
# convertir categórica a numérica ordinal "ordinal encoding"
def ordinal_encoding(df, columna_categorica, lista_ordenada_valores):
    '''
    Docstring para ordinal_encoding
    :param df: dataframe
    :param columna_categorica: columa que quiero convertir de categórica a numérica
    :param lista_ordenada_valores: lista ordenada de los valores de la categórica que quiero pasar a números
    '''
    ordinal_encoder = OrdinalEncoder(categories= [lista_ordenada_valores])
    df[columna_categorica] = ordinal_encoder.fit_transform(df[[columna_categorica]])
    
    return df


# convertir categórica a varias columnas de 1 y 0 (tantas columnas como valores tome) "one hot encoding"
def one_hot_encoding(df, columna_categorica, tipo = int):
    '''
    Docstring para one_hot_encoding
    
    :param df: dataframe
    :param columna_categorica: columna categorica que quiero convertir a dummie
    :param tipo: int o bool
    '''
    df = pd.get_dummies(df, columns = [columna_categorica], dtype = tipo)
    return df



# 4. Tratamiento numéricas

In [ ]:
# cuando hay variables que toman valores muy pequeños y otras variables valores muy grandes, debemos escalarlas o normalizarlas para que unas
# no afecten más que otras al modelo

# primero ver las distribuciones:
# si tiene heavy tail: transformacion log
# si el rango de valores es muy grande pero está distribuida de forma homogenea min max scalin
# si parece una distribución normal la normalizo a N(0, 1)

# min-max scaling "normalización entre 0 y 1 o -1 y 1 (mejor para redes neuronales)" 
# cuando los rangos de valores son muy grandes o hay outliers...
def min_max_escaling(df, lista_columas_numericas, rango=(0, 1)):
    min_max_scaler = MinMaxScaler(feature_range=rango)
    df[lista_columas_numericas] = min_max_scaler.fit_transform(df[lista_columas_numericas]), columns= lista_columas_numericas
    return df

# estandarización (lo convierte a normales de media 0 y desviación 1)
def estandarizacion(df, lista_columas_numericas):
    std_scaler = StandardScaler()
    df[lista_columas_numericas] = std_scaler.fit_transform(df[lista_columas_numericas])
    return df

# transformado de distribuciones con log (cuando una variable tiene heavy tail una cola muy larga y los valores concentrados en un lado)
fig, axs = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
train_set["total_rooms"].hist(ax=axs[0], bins=50)
train_set["total_rooms"].apply(np.log).hist(ax=axs[1], bins=50)
axs[0].set_xlabel("total rooms")
axs[1].set_xlabel("Log of total rooms")
axs[0].set_ylabel("Number of districts")
plt.show()

def transformacion_log_numericas(df, lista_columas_numericas_heavy_tail):
    min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
    for col in lista_columas_numericas_heavy_tail: 
        df[f"log_{col}"] = df[col].apply(np.log)
        df[col] = min_max_scaler.fit_transform(df[[f"log_{col}"]])
    return df




# 5. Proceso modelado

In [ ]:
# ejecutamos modelos
lin_reg = LinearRegression()
tree_reg = DecisionTreeRegressor(random_state = 42) # Necesita una inicialización aleatoria y la semilla permite que siempre sea la misma
log_class = LogisticRegression()
rforest_class = RandomForestClassifier(n_estimators= 20, max_depth= 3, max_leaf_nodes= 5)

# primero separamos la variable target del dataset de train y además solo nos quedamos con las features:
def modelado_y_prediccion_y_mse_del_train(train_df, features, target, modelo):
    '''
    modelo puede ser LinearRegression(), DecisionTreeRegressor(random_state = 42)...
    '''
    X = train_df[features].copy()
    y= train_df[target]
    modelo.fit(X, y)
    train_pred = modelo.predict(X)
    return X, y, modelo, train_pred

# tomando lo que suelta la función anterior
def mse_del_train(y, train_pred):
    mse_train = mean_squared_error(y, train_pred)
    print(f"El error cuadrático medio de la predicción del target del train del modelo es {sqrt(mse_train)}")
    return mse_train


# otra forma de ver el error es con la accuracy
def accuracy(y, train_pred):
    acc_train = accuracy_score(y, train_pred)
    print(f"El accuracy de la predicción del target del train del modelo es {acc_train}")
    return acc_train

# 6. Proceso evaluación y ajuste

In [ ]:
#comprobamos que no hay sobreajuste en el modelo 
# haciendo cross validation y viendo que el error mse del modelo y el error del cross validation sean parecidos (si son diferentes hay sobreajuste)
def comparacion_errores_mse_cross_validation(X, y, train_pred, modelo, k = 10):
    '''
    X, y y train_pred lo devuelve la función anterior
    modelo puede ser LinearRegression(), DecisionTreeRegressor(random_state = 42)...
    k es el número de conjuntos en los que divido el train para hacer de nuevo el modelo por separado
    '''

    mse_train = mean_squared_error(y, train_pred)
    rmses = -cross_val_score(modelo, X, y, scoring="neg_root_mean_squared_error", cv=k) 
    print(f"El error cuadrático medio del modelo es {sqrt(mse_train)}")
    print(f"La media de los errores de los modelos al separar en {k} conjuntos de entrenamiento es {pd.Series(rmses).mean()}")


def comparacion_errores_accuracy_cross_validation(X, y, train_pred, modelo, k = 10):
    '''
    X, y y train_pred lo devuelve la función anterior
    modelo puede ser LinearRegression(), DecisionTreeRegressor(random_state = 42)...
    k es el número de conjuntos en los que divido el train para hacer de nuevo el modelo por separado
    '''

    acc_train = accuracy_score(y, train_pred)
    rmses = cross_val_score(modelo, X, y, scoring="accuracy", cv=k) 
    print(f"La accuracy del modelo es {acc_train}")
    print(f"La media de las accuracy de los modelos al separar en {k} conjuntos de entrenamiento es {pd.Series(rmses).mean()}")



# luego hacemos las mismas transformaciones al test que le hicimos al train

# y por último, evaluamos con el modelo escogido sobre el test, y si el mse de mantiene cercano al mse del train, el modelo es bueno
def prediccion_y_mse_del_test(test_df, features, target, modelo):
    '''
    modelo puede ser LinearRegression(), DecisionTreeRegressor(random_state = 42)...
    '''

    X_test = test_df[features].copy()
    y_test = test_df[target]
    test_pred = modelo.predict(X_test)
    mse_test = mean_squared_error(y_test, test_pred)
    return print(f"El error cuadrático medio de la predicción del target del test del modelo es {sqrt(mse_test)}")



